Based on 4chan NovelAILeaks (naifu)[src](https://boards.4channel.org/g/thread/89095460#p89097704)

Credit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb

In [ ]:
#@title ### 0. GPUの動作状況を確認

!nvidia-smi

In [ ]:
#@title ### 1. Novel AI APIバックエンド・モデルのダウンロード
#@markdown ダウンロード速度が遅い場合は、再起動してください

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. 依存関係のインストール
#@markdown インストール完了まで、しばらくお待ちください

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

In [ ]:
#@title ### 3. WAIFU起動
#@markdown 出力マッピングアドレスの（末尾が `trycloudflare.com` / `bore.pub` ）アクセスするだけです。
#@markdown - モデルがロードされるまで（「Application startup complete」が表示されるまで）、アクセスはお控えください。
#@markdown - Cloudflare が提供するサービスでは、リクエストタイムアウトが発生することがあり、これを bore tunnel で置き換えることができます

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. (オプション) 7GBの`animefull-latest`モデルで実行。
#@markdown デフォルトで4GBのanimefull-final-prunedモデルを使用します。7GBのanimefull-latestモデルを使いたい場合は、こちらを実行します。

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969